In [1]:
a = [
            "정치", "선거", "정부", "입법", "외교", "정당", "대통령직", "국회", "정책", "국가 안보",
            "투표", "경제", "주식 시장", "경제 상황", "금융", "세금", "무역", "투자", "부동산", "예산",
            "인플레이션", "실업", "사회", "교육", "범죄", "복지", "건강", "노동", "이민", "인권",
            "환경", "교통", "사건사고", "문화", "예술", "음악", "영화", "문학", "전시회", "축제",
            "시각 예술", "극장", "전통", "패션", "과학", "기술", "연구", "우주", "환경 과학", "의학",
            "발명품", "데이터", "로봇 공학", "인공 지능", "에너지", "국제 문제", "외교 정책", "글로벌 관계", "세계 경제", "국제법",
            "해외 이벤트", "글로벌 이슈", "유엔", "국제 컨퍼런스", "해외 문화", "국경", "스포츠", "올림픽", "축구", "야구",
            "농구", "테니스", "골프", "마라톤", "수영", "피트니스", "모터스포츠", "인터넷", "소프트웨어", "하드웨어",
            "스마트폰", "게임", "사이버 보안", "앱", "클라우드 컴퓨팅", "빅 데이터", "가상 현실", "영양", "질병", "의료",
            "운동", "심리학", "웰니스", "건강 보험", "예방", "치료", "건강 식품", "여행", "여행 팁", "호텔"
        ]

In [3]:
a = ["Politics", "Economics", "Social Issue", "Culture & Arts", "Natural Science", "International Affairs", "Technology & Internet", "Health & Wellness", "Travel"]

In [5]:
c = {
  "Politics": [
    "Politics", "Elections", "Government", "Legislation", "Diplomacy", "Political Parties", "Presidency", "Parliament", "Policies", "National Security", "Voting"
  ],
  "Economics": [
    "Economy", "Stock Market", "Economic Conditions", "Finance", "Taxes", "Trade", "Investments", "Real Estate", "Budget", "Inflation", "Unemployment"
  ],
  "Social Issue": [
    "Society", "Education", "Crime", "Welfare", "Health", "Labor", "Immigration", "Human Rights", "Environment", "Transportation", "Incidents"
  ],
  "Culture & Arts": [
    "Culture", "Arts", "Music", "Movies", "Literature", "Exhibitions", "Festivals", "Visual Arts", "Theater", "Tradition", "Fashion"
  ],
  "Natural Science": [
    "Science", "Technology", "Research", "Space", "Environmental Science", "Medicine", "Inventions", "Data", "Robotics", "Artificial Intelligence", "Energy"
  ],
  "International Affairs": [
    "International Issues", "Diplomatic Policies", "Global Relations", "World Economy", "International Law", "Overseas Events", "Global Issues", "United Nations", "International Conferences", "Foreign Cultures", "Borders"
  ],
  "Sports": [
    "Sports", "Olympics", "Soccer", "Baseball", "Basketball", "Tennis", "Golf", "Marathons", "Swimming", "Fitness", "Motorsports"
  ],
  "Technology & Internet": [
    "Internet", "Software", "Hardware", "Smartphones", "Games", "Cybersecurity", "Apps", "Cloud Computing", "Big Data", "Virtual Reality"
  ],
  "Health & Wellness": [
    "Nutrition", "Diseases", "Medicine", "Exercise", "Psychology", "Wellness", "Health Insurance", "Prevention", "Treatment", "Healthy Foods"
  ],
  "Travel": [
    "Travel", "Travel Tips", "Hotels"
  ]
}


In [9]:
c = {
  "정치": [
    "정치", "선거", "정부", "입법", "외교", "정당", "대통령직", "국회", "정책", "국가 안보", "투표"
  ],
  "경제": [
    "경제", "주식 시장", "경제 상황", "금융", "세금", "무역", "투자", "부동산", "예산", "인플레이션", "실업"
  ],
  "사회 문제": [
    "사회", "교육", "범죄", "복지", "건강", "노동", "이민", "인권", "환경", "교통", "사건사고"
  ],
  "문화 및 예술": [
    "문화", "예술", "음악", "영화", "문학", "전시회", "축제", "시각 예술", "극장", "전통", "패션"
  ],
  "자연 과학": [
    "과학", "기술", "연구", "우주", "환경 과학", "의학", "발명품", "데이터", "로봇 공학", "인공 지능", "에너지"
  ],
  "국제 문제": [
    "국제 문제", "외교 정책", "글로벌 관계", "세계 경제", "국제법", "해외 이벤트", "글로벌 이슈", "유엔", "국제 컨퍼런스", "해외 문화", "국경"
  ],
  "스포츠": [
    "스포츠", "올림픽", "축구", "야구", "농구", "테니스", "골프", "마라톤", "수영", "피트니스", "모터스포츠"
  ],
  "기술 및 인터넷": [
    "인터넷", "소프트웨어", "하드웨어", "스마트폰", "게임", "사이버 보안", "앱", "클라우드 컴퓨팅", "빅 데이터", "가상 현실"
  ],
  "건강 및 웰빙": [
    "영양", "질병", "의료", "운동", "심리학", "웰니스", "건강 보험", "예방", "치료", "건강 식품"
  ],
  "여행": [
    "여행", "여행 팁", "호텔"
  ]
}


In [10]:
count = 0
for key, value in c.items():
    print(len(value))
    count += len(value)

11
11
11
11
11
11
11
10
10
3


In [11]:
print(count)

100


In [19]:
data = pd.read_csv("C:/Users/cyon1/Desktop/LLAMA/Articles.csv", encoding='latin1')

In [20]:
data

,Article,Date,Heading,NewsType
0,KARACHI: The Sindh government has decided to b...,1/1/2015,sindh govt decides to cut public transport far...,business
1,HONG KONG: Asian markets started 2015 on an up...,1/2/2015,asia stocks up in new year trad,business
2,HONG KONG: Hong Kong shares opened 0.66 perce...,1/5/2015,hong kong stocks open 0.66 percent lower,business
3,HONG KONG: Asian markets tumbled Tuesday follo...,1/6/2015,asian stocks sink euro near nine year,business
4,NEW YORK: US oil prices Monday slipped below $...,1/6/2015,us oil prices slip below 50 a barr,business
...,...,...,...,...
2687,strong>DUBAI: Dubai International Airport and ...,3/25/2017,Laptop ban hits Dubai for 11m weekend traveller,business
2688,"strong>BEIJING: Former Prime Minister, Shaukat...",3/26/2017,Pak China relations not against any third coun...,business
2689,strong>WASHINGTON: Uber has grounded its fleet...,3/26/2017,Uber grounds self driving cars after accid,business
2690,strong>BEIJING: The New Development Bank plans...,3/27/2017,New Development Bank plans joint investments i...,business


In [30]:
desc = data["Article"][1]

HONG KONG: Asian markets started 2015 on an upswing in limited trading on Friday, with mainland Chinese stocks surging in Hong Kong on speculation Beijing may ease monetary policy to boost slowing growth.Hong Kong rose 1.07 percent, closing 252.78 points higher at 23857.82.Seoul closed up 0.57 percent, rising 10.85 points to 1,926.44, while Sydney gained 0.46 percent, or 24.89 points, to close at 5,435.9.Singapore edged up 0.19 percent, gaining 6.39 points to 3,371.54.Markets in mainland China, Japan, Taiwan, New Zealand, the Philippines, and Thailand remained closed for holidays.With mainland bourses shut until January 5, shares in Chinese developers and financial companies surged in Hong Kong, stoked by hopes that Beijing could ease monetary policy to support lagging growth in the world´s second-largest economy.China Vanke, the country´s biggest developer by sales, leapt 10.8 percent and the People´s Insurance Company (Group) of China Ltd. was up 5.51 percent in afternoon trading.Tra